In [1]:
import torch
from pprint import pprint
from tqdm import tqdm
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("models/gpt2/ta_encode/multiwoz",
                                            padding_side="left", truncation_side="left")
model = GPT2LMHeadModel.from_pretrained("models/gpt2/ta_encode/multiwoz")

In [3]:
datasets = load_dataset("json", data_files={
        "train": "data/multiwoz/train/encoded.json",
        "valid": "data/multiwoz/dev/encoded.json",
        "test": "data/multiwoz/test/encoded.json",
    })

Using custom data configuration default-9ff1588b4a0715cd
Reusing dataset json (/home/jader/.cache/huggingface/datasets/json/default-9ff1588b4a0715cd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from utils import dbPointer
dbPointer.queryResultVenues("train", {
                        "departure" :"broxbourne",
                        "destination": "cambridge",
                        "leaveat": "10:15"
                        }, True)[0]

select * from train where  departure='broxbourne' and destination='cambridge' and leaveat > '10:15'


('TR9199',
 '11:32',
 'monday',
 'broxbourne',
 'cambridge',
 '60 minutes',
 '10:32',
 '17.90 pounds')

In [42]:
sizencode = 256
predicted = {}
for batch in tqdm(datasets["test"]):
    did = batch["id"].lower().rstrip(".json")
    utterances = batch["text"].split("<sos_r>")
    predicted[did] = []
    responses = []
    for i in range(len(utterances)-1):
        example = "<sos_r>".join(utterances[:i+1])[-sizencode:]
        responses.append(example)
    encode = tokenizer(responses, return_tensors="pt", truncation=True,
                        padding=True, max_length=sizencode)
    gen = model.generate(
        **encode,
        max_new_tokens=80,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.encode("<eos_r>")[0]
    )
    for response in gen:
        response = tokenizer.decode(response)
        predicted[did].append({
            "response": response.split("<sos_r>")[-1].split("<eos_r>")[0].strip(),
            # TODO
            # "state": response.split("<sos_b>")[-1].split("<eos_b>")[0].strip(),
            })

100%|██████████| 1000/1000 [2:01:49<00:00,  7.31s/it]     


In [43]:
from mwzeval.metrics import Evaluator
e = Evaluator(bleu=True, success=False, richness=True)
results = e.evaluate(predicted)
print(results)

{'bleu': {'mwz22': 14.149226756890032}, 'success': None, 'richness': {'entropy': 6.044637005567655, 'cond_entropy': 1.9912459899261963, 'avg_lengths': 18.25881714595768, 'msttr': 0.6195839524517073, 'num_unigrams': 492, 'num_bigrams': 2244, 'num_trigrams': 4541}, 'dst': None}


In [4]:
model = GPT2LMHeadModel.from_pretrained("models/gpt2/multiwoz")

OSError: We couldn't connect to 'https://huggingface.co/' to load this model and it looks like models/gpt2/multiwoz is not the path to a directory conaining a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
sizencode = 256
predicted = {}
for batch in tqdm(datasets["test"]):
    did = batch["id"].lower().rstrip(".json")
    utterances = batch["text"].split("<sos_r>")
    predicted[did] = []
    responses = []
    for i in range(len(utterances)-1):
        example = "<sos_r>".join(utterances[:i+1])[-sizencode:]
        responses.append(example)
    encode = tokenizer(responses, return_tensors="pt", truncation=True,
                        padding=True, max_length=sizencode)
    gen = model2.generate(
        **encode,
        max_new_tokens=80,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.encode("<eos_r>")[0]
    )
    for response in gen:
        response = tokenizer.decode(response)
        predicted[did].append({
            "response": response.split("<sos_r>")[-1].split("<eos_r>")[0].strip(),
            # TODO
            # "state": response.split("<sos_b>")[-1].split("<eos_b>")[0].strip(),
            })

In [ ]:
from mwzeval.metrics import Evaluator
e = Evaluator(bleu=True, success=False, richness=True)
results = e.evaluate(predicted)
print(results)

In [41]:
text = "<sos_u> I need a train going to Cambridge that will depart after 10:15 from broxbourne. <eos_u>"
for i in range(80):
    example = tokenizer(text, return_tensors="pt")
    logits = model(**example).logits[0]
    text += tokenizer.decode(torch.argmax(logits[-1]))
print(text)

<sos_u> I need a train going to Cambridge that will depart after 10:15 from broxbourne. <eos_u><sos_b>[find_train]train-departure broxbourne train-destination cambridge train-leaveat 10:15<eos_b><sos_a>[train-inform] choice many [train-request] day?<eos_a><sos_r>There are <value_choice> trains that meet your criteria. What day would you like to travel?<eos_r><sos_u>I would like to leave on Tuesday.<eos_u><sos_b>[find_train]train-day t
